# Synthetic Multimodal Question Generation

In [1]:
%load_ext autoreload
%autoreload 2

import os
import re
from dotenv import load_dotenv
load_dotenv()

True

## 1. Read & Preprocess PDF file

---

### Split the PDFs into individual pages

In [2]:
import shutil, random
import openai
from unstructured.cleaners.core import clean_bullets, clean_extra_whitespace, remove_punctuation
from langchain_community.document_loaders import UnstructuredFileLoader, UnstructuredMarkdownLoader, UnstructuredAPIFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader, UnstructuredCSVLoader
from langchain_ollama import ChatOllama
from util.common_utils import get_language_code

raw_data_dir = "../raw_data"

splitted_raw_data_dir = "splitted_raw_data"
file_path = f"{raw_data_dir}/pdf/vi-vneid.pdf"

DOMAIN = "VNeID Application Usage Guide in Vietnam"
LANGUAGE = "Vietnamese" # You can change your language here. e.g., "Korean", "Japanese", "Chinese"
LANGUAGE_CODE = get_language_code(LANGUAGE)
print(f"Domain: {DOMAIN}, Language: {LANGUAGE}, Language Code: {LANGUAGE_CODE}")

Domain: VNeID Application Usage Guide in Vietnam, Language: Vietnamese, Language Code: vi


In [3]:
# (Optional) Only use a poration of the PDF documents for testing. If there are a lot of pages or partial processing is required, cut and save only some pages.
import fitz

# Open the first PDF document
doc1 = fitz.open(file_path)
split_pages = [(7, 15)]

for idx, s in enumerate(split_pages):
    # Create a new empty PDF document
    doc2 = fitz.open()

    # Insert the first 2 pages of doc1 into doc2
    doc2.insert_pdf(doc1, from_page=s[0], to_page=s[1])

    # Save the modified document
    doc2.save(f"{raw_data_dir}/part{idx}.pdf")

In [4]:
from util.common_utils import delete_folder_and_make_folder
from util.preprocess import remove_short_sentences, remove_small_images, split_pdf
from collections import defaultdict

file_path = f"{raw_data_dir}/part0.pdf"
file_path

'../raw_data/part0.pdf'

In [5]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_ollama import ChatOllama

max_tokens = 1024
llm = ChatOllama(
    model="gemma3:4b-it-qat",
    temperature=0,
    base_url="http://localhost:11434",
    num_predict=max_tokens
)

system_prompt = "You are an assistant tasked with reading content of image"
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)
human_prompt = [
    {
        "type": "image_url",
        "image_url": {
            "url": "data:image/png;base64," + "{image_base64}",
        },
    },
    {
        "type": "text",
        "text": f"Given image, give a concise summary in {LANGUAGE}. Don't insert any XML tag such as <text> and </text> when answering."
    },
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message_template,
        human_message_template
    ]
)

summarize_chain = prompt | llm | StrOutputParser()

In [6]:
import fitz  # PyMuPDF
import pdfplumber
import base64
import io
from PIL import Image
from transformers import pipeline

def pdf_to_data(pdf_path, min_text_leng):
    result = {
        "texts": [],
        "images": [],
        "tables": []
    }

    # Open PDF
    doc = fitz.open(pdf_path)
    plumber_pdf = pdfplumber.open(pdf_path)

    for page_num, (page_fitz, page_plumber) in enumerate(zip(doc, plumber_pdf.pages)):
        # --- Extract Text ---
        text = page_fitz.get_text()
        if text.strip():
            text = re.sub(r'\n{2,}', '\n', text.strip())
            if len(text) >= min_text_leng:
                result["texts"].append(text)
            

        # --- Extract Images ---
        image_list = page_fitz.get_images(full=True)
        for img_index, img in enumerate(image_list):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]

            # Encode image to base64
            image_base64 = base64.b64encode(image_bytes).decode('utf-8')
            summary = summarize_chain.invoke(image_base64)
            caption = f"Image_{len(result['images'])+1}, Page_{page_num}"
            # print("\n ==================")
            # print(f"{caption=}")
            print(f"{summary=}")
            result["images"].append({
                'image_base64': image_base64,
                'verbalisation': summary,
                'caption': caption
            })

        # --- Extract Tables ---
        tables = page_plumber.extract_tables()
        for table in tables:
            if not table:
                continue
            md_table = '\n'.join(' | '.join(item for item in row if item and item is not None) for row in table)
            if len(md_table) > 10:
                result["tables"].append(md_table)

    doc.close()
    plumber_pdf.close()

    return result


/home/dino/miniconda3/envs/qa-gen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
%%time
data = pdf_to_data(file_path, min_text_leng=200)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


summary='Hình ảnh mô tả về việc lập danh mục điện tử cho các cấp chính quyền, bao gồm cả việc tích hợp thông tin từ Cơ sở dữ liệu Quốc gia và việc di chuyển dữ liệu cho sinh hoạt xã hội.'
summary='Ảnh hiển thị màn hình điện thoại với hiển thị giờ, ngày, nhiệt độ (34°C) và địa điểm Hà Nội, cùng với các biểu tượng của Google, Cửa hàng Google Play, cuộc gọi và các biểu tượng khác.'
summary='Hình ảnh hiển thị ứng dụng tên là VNeID, có thể liên quan đến các dịch vụ công an.'
summary='Hình ảnh hiển thị một ứng dụng dành cho trẻ em, có tên là "Ứng dụng dành cho trẻ em". Ứng dụng này có đánh giá 2.9 sao, dung lượng 4.0 MB và phù hợp cho trẻ từ 3 tuổi trở lên.'
summary='Ứng dụng này giúp bạn xác thực danh tính và bảo vệ thông tin cá nhân, đồng thời cung cấp các tính năng mới cập nhật vào ngày 10 tháng 11 năm 2021.'
summary='Hình ảnh hiển thị một người lính và một cô gái, có thể là một cặp đôi hoặc thành viên gia đình.'
summary='Hình ảnh hiển thị một biểu tượng ứng dụng điện thoại với các biểu t

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


summary='Hình ảnh hiển thị thông báo về việc cập nhật danh điện tử, đồng bộ thông tin và hỗ trợ công dân trong quá trình đăng ký.'
CPU times: user 1.32 s, sys: 52.1 ms, total: 1.37 s
Wall time: 44min 34s


In [8]:
import json
# Ghi ra file JSON
with open('preprocessing_result_3.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [9]:
import json
with open('preprocessing_result_3.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

print(data)

{'texts': ['7 \n \n1. \nGIỚI THIỆU TỔNG QUAN \n1.1 \nĐối tượng sử dụng \n- \nDùng cho công dân Việt Nam có căn cước công dân gắn chíp thực hiện \nđăng ký tài khoản Định danh diện tử \n1.2 \n Mô tả tài liệu \nNội dung tài liệu bao gồm các phần sau: \n1. \nMục A: Giới thiệu tổng quan \n2. \nMục B: Hướng dẫn các chức năng hệ thống có trên APP cho người dân sử \ndụng. \n1.3 \n Thuật ngữ viết tắt \nSTT \nThuật ngữ \nÝ nghĩa \n1 \nCCCD \nCăn cước công dân \n2 \nSĐT \nSố điện thoại \n3 \nNSD \nNgười sử dụng \n1.4 Cấu trúc hệ thống \nSau khi đăng nhập vào hệ thống, màn hình trang chủ hiển thị giao diện như hình.', '8 \n \n \nHình 1 Giao diện trang chủ mức 0 \n1.5 Chức năng chung \n-  Đăng nhập \n-  Đăng ký mức 0 \n-  Quên mật khẩu \n-  Đăng ký mức 1 \n-  Kích hoạt tài khoản \n-  Trang chủ  \n-  Ví giấy tờ  \n-  Tab Cá nhân \n- Đổi tài khoản \n- Thông báo lưu trú', '9 \n \n2. HƯỚNG DẪN SỬ DỤNG \n2.1 Hướng dẫn cài đặt  \n2.1.1  Đối với hệ điều hành Android \nCài đặt ứng dụng từ CH Play \n-    Bư

In [11]:
print(data['texts'][0])

7 
 
1. 
GIỚI THIỆU TỔNG QUAN 
1.1 
Đối tượng sử dụng 
- 
Dùng cho công dân Việt Nam có căn cước công dân gắn chíp thực hiện 
đăng ký tài khoản Định danh diện tử 
1.2 
 Mô tả tài liệu 
Nội dung tài liệu bao gồm các phần sau: 
1. 
Mục A: Giới thiệu tổng quan 
2. 
Mục B: Hướng dẫn các chức năng hệ thống có trên APP cho người dân sử 
dụng. 
1.3 
 Thuật ngữ viết tắt 
STT 
Thuật ngữ 
Ý nghĩa 
1 
CCCD 
Căn cước công dân 
2 
SĐT 
Số điện thoại 
3 
NSD 
Người sử dụng 
1.4 Cấu trúc hệ thống 
Sau khi đăng nhập vào hệ thống, màn hình trang chủ hiển thị giao diện như hình.


## Step 1: Sample Seed Source

Để giải thích chi tiết về cách lấy mẫu nguồn (seed source) từ tập hợp các nguồn $S$ trong bước 1 của phương pháp Synthetic Multimodal Question Generation (SMMQG), chúng ta cần tập trung vào công thức và quy trình được mô tả trong bài báo. Dưới đây là phân tích chi tiết:

### **Bối cảnh của bước 1: Lấy mẫu nguồn khởi đầu (Sample Seed Source)**

Mục tiêu của bước 1 là chọn một nguồn khởi đầu $s_{seed} \in S$ từ tập hợp các nguồn đa phương thức $S$, bao gồm các đoạn văn bản (text passages), bảng (tables), và hình ảnh (images). Nguồn khởi đầu này sẽ được sử dụng để bắt đầu quá trình tạo câu hỏi, và việc chọn nguồn phù hợp là rất quan trọng để đảm bảo rằng các câu hỏi được tạo ra có tính liên kết và ý nghĩa.

Cách đơn giản nhất để chọn $s_{seed}$ là lấy mẫu ngẫu nhiên đồng đều (uniform sampling) từ $S$. Tuy nhiên, bài báo chỉ ra rằng phương pháp này không tối ưu vì nhiều nguồn trong $ S $ có thể là "ngoại lệ" (outliers), tức là không liên quan chặt chẽ đến các nguồn khác hoặc không phản ánh các chủ đề chính trong tài liệu. Những nguồn như vậy khó có thể được sử dụng để tạo ra các câu hỏi đa nguồn có ý nghĩa, vì chúng thiếu sự liên kết ngữ nghĩa với các nguồn khác.

Để khắc phục vấn đề này, SMMQG sử dụng một phương pháp lấy mẫu có trọng số (weighted sampling), trong đó xác suất chọn một nguồn được điều chỉnh dựa trên mức độ liên quan của nguồn đó với các nguồn khác trong $S$. Phương pháp này được mô tả thông qua công thức xác suất lấy mẫu và cách tính trọng số.

### **Công thức xác suất lấy mẫu**

Xác suất để chọn nguồn $s_i \in S$ làm nguồn khởi đầu $s_{\text{seed}}$ được định nghĩa như sau:

$$
p_{s_i} = \frac{\exp \left(-\beta w_i\right)}{\sum_j \exp \left(-\beta w_j\right)}
$$

Trong đó:
- $ p_{s_i} $: Xác suất chọn nguồn $ s_i $ làm $ s_{\text{seed}} $.
- $ w_i $: Trọng số của nguồn $ s_i $, phản ánh mức độ "cô lập" của nguồn này so với các nguồn khác trong $ S $.
- $ \beta $: Tham số nhiệt độ (temperature parameter), kiểm soát mức độ ưu tiên các nguồn có trọng số thấp. Trong bài báo, $ \beta = 0.1 $ được chọn dựa trên kiểm tra thủ công (manual inspection) của kết quả đầu ra.
- Phần mẫu $ \sum_j \exp \left(-\beta w_j\right) $: Chuẩn hóa tổng các trọng số để đảm bảo tổng xác suất bằng 1.

Công thức này sử dụng dạng phân phối Boltzmann (hay softmax với trọng số âm), trong đó các nguồn có trọng số $ w_i $ nhỏ hơn (tức là liên quan hơn đến các nguồn khác) sẽ có xác suất được chọn cao hơn.

### **Cách tính trọng số $ w_i $**

Trọng số $ w_i $ của nguồn $ s_i $ được tính dựa trên khoảng cách trung bình trong không gian nhúng (embedding space) giữa $ s_i $ và các nguồn láng giềng gần nhất của nó. Công thức cụ thể là:

$$
w_i = \frac{1}{k_{\text{seed}}} \sum_{s_j \in k_{\text{seed}} \text{nn}(s_i)} \text{dist} \left( E(s_i), E(s_j) \right)
$$

Trong đó:
- $ k_{\text{seed}} $: Số lượng láng giềng gần nhất được xem xét. Trong bài báo, $ k_{\text{seed}} = 5 $.
- $ k_{\text{seed}} \text{nn}(s_i) $: Tập hợp $ k_{\text{seed}} $ nguồn gần nhất với $ s_i $ trong không gian nhúng, được xác định bằng cách sử dụng mô hình nhúng $ E $.
- $ E(s_i) $: Vector nhúng của nguồn $ s_i $, được tạo bởi mô hình nhúng (trong bài báo, sử dụng mô hình **E5-Large**).
- $ \text{dist} \left( E(s_i), E(s_j) \right) $: Khoảng cách cosine (cosine distance) giữa các vector nhúng của $ s_i $ và $ s_j $, được tính như:

$$
\text{dist} \left( E(s_i), E(s_j) \right) = 1 - \frac{E(s_i) \cdot E(s_j)}{\|E(s_i)\| \|E(s_j)\|}
$$

- Phần tử $ \frac{1}{k_{\text{seed}}} $: Chuẩn hóa bằng cách lấy trung bình các khoảng cách.

Ý nghĩa của $ w_i $:
- Nếu $ w_i $ nhỏ, điều đó có nghĩa là $ s_i $ có các láng giềng gần trong không gian nhúng, tức là nó liên quan chặt chẽ đến các nguồn khác và có khả năng thuộc về một chủ đề chính trong tài liệu.
- Nếu $ w_i $ lớn, $ s_i $ có xu hướng là một nguồn "cô lập" (outlier), ít liên quan đến các nguồn khác, và do đó ít phù hợp để làm nguồn khởi đầu.

### **Quy trình lấy mẫu nguồn khởi đầu**

1. **Tạo nhúng cho các nguồn**:
   - Mỗi nguồn $ s_i \in S $ (có thể là đoạn văn, bảng, hoặc hình ảnh) được chuyển thành vector nhúng $ E(s_i) $ bằng mô hình nhúng **E5-Large**. 
   - Đối với hình ảnh, bài báo sử dụng mô tả văn bản (image verbalisation) và chú thích (caption) để tạo nhúng, vì E5-Large là mô hình dựa trên văn bản.

2. **Tính khoảng cách đến láng giềng gần nhất**:
   - Với mỗi nguồn $ s_i $, tìm $ k_{\text{seed}} = 5 $ nguồn gần nhất trong $ S $ (không bao gồm chính $ s_i $) dựa trên khoảng cách cosine giữa các vector nhúng.
   - Tính khoảng cách cosine trung bình giữa $ E(s_i) $ và các vector nhúng của $ k_{\text{seed}} $ láng giềng gần nhất để có được $ w_i $.

3. **Tính xác suất lấy mẫu**:
   - Sử dụng công thức $ p_{s_i} $ với $ \beta = 0.1 $ để tính xác suất chọn từng nguồn $ s_i $.
   - Tham số $ \beta $ nhỏ (0.1) làm cho sự khác biệt giữa các xác suất $ p_{s_i} $ được khuếch đại, ưu tiên mạnh mẽ các nguồn có $ w_i $ nhỏ (tức là các nguồn liên quan hơn).

4. **Lấy mẫu nguồn**:
   - Dựa trên phân phối xác suất $ \{ p_{s_i} \} $, chọn ngẫu nhiên một nguồn $ s_{\text{seed}} $ từ $ S $. Các nguồn có xác suất cao hơn (tương ứng với $ w_i $ thấp) sẽ có khả năng được chọn cao hơn.

### **Ý nghĩa của phương pháp**

- **Tránh các nguồn ngoại lệ**: Bằng cách ưu tiên các nguồn có $ w_i $ nhỏ, phương pháp đảm bảo rằng $ s_{\text{seed}} $ có khả năng liên quan đến các nguồn khác, giúp dễ dàng hơn trong việc tìm các nguồn liên quan ở bước 3 (Retrieve Candidate Sources).
- **Tăng tính liên kết ngữ nghĩa**: Việc chọn nguồn khởi đầu dựa trên sự tương đồng ngữ nghĩa đảm bảo rằng các câu hỏi đa nguồn được tạo ra ở các bước sau sẽ có tính liên kết và tập trung vào các chủ đề chính của tài liệu.
- **Tính linh hoạt**: Công thức này có thể được điều chỉnh bằng cách thay đổi $ \beta $ hoặc $ k_{\text{seed}} $, cho phép kiểm soát mức độ ưu tiên các nguồn liên quan hoặc đa dạng hóa các nguồn được chọn.



In [13]:
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_distances

In [19]:
# Hàm tạo nhúng cho danh sách các nguồn
def get_embeddings(sources, model_name="BAAI/bge-m3"):
    # Tải tokenizer và mô hình
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    model.eval()

    # Chuyển các nguồn thành nhúng
    embeddings = []
    with torch.no_grad():
        for source in sources:
            # Tokenize và chuyển thành tensor
            inputs = tokenizer(source, return_tensors="pt", padding=True, truncation=True, max_length=512)
            # Tạo nhúng
            outputs = model(**inputs)
            # Lấy nhúng từ [CLS] token hoặc trung bình các token
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
            embeddings.append(embedding)
    
    return np.array(embeddings)

In [20]:
# Hàm tính trọng số w_i cho mỗi nguồn
def compute_weights(embeddings, k_seed=5):
    # Tính khoảng cách cosine giữa tất cả các cặp nhúng
    distances = cosine_distances(embeddings)
    
    # Tính trọng số w_i
    weights = []
    for i in range(len(embeddings)):
        # Lấy k_seed láng giềng gần nhất (không tính chính nó)
        dist_i = distances[i]
        nearest_indices = np.argsort(dist_i)[1:k_seed+1]  # Bỏ qua chính i
        nearest_distances = dist_i[nearest_indices]
        # Tính trung bình khoảng cách
        w_i = np.mean(nearest_distances)
        weights.append(w_i)
    
    return np.array(weights)

In [21]:
# Hàm tính xác suất lấy mẫu p_{s_i}
def compute_sampling_probabilities(weights, beta=0.1):
    # Tính exp(-\beta * w_i)
    exp_weights = np.exp(-beta * weights)
    # Chuẩn hóa để tổng xác suất bằng 1
    probabilities = exp_weights / np.sum(exp_weights)
    return probabilities

In [22]:
# Hàm lấy mẫu nguồn khởi đầu
def sample_seed_source(sources, model_name="BAAI/bge-m3", k_seed=5, beta=0.1):
    # Bước 1: Tạo nhúng cho các nguồn
    embeddings = get_embeddings(sources, model_name)
    
    # Bước 2: Tính trọng số w_i
    weights = compute_weights(embeddings, k_seed)
    
    # Bước 3: Tính xác suất lấy mẫu
    probabilities = compute_sampling_probabilities(weights, beta)
    
    # Bước 4: Lấy mẫu ngẫu nhiên dựa trên xác suất
    seed_index = np.random.choice(len(sources), p=probabilities)
    seed_source = sources[seed_index]
    
    return seed_source, seed_index, probabilities

In [27]:
source = data['texts'] + data['tables'] + [item['verbalisation'] for item in data['images']]

In [28]:
source

['7 \n \n1. \nGIỚI THIỆU TỔNG QUAN \n1.1 \nĐối tượng sử dụng \n- \nDùng cho công dân Việt Nam có căn cước công dân gắn chíp thực hiện \nđăng ký tài khoản Định danh diện tử \n1.2 \n Mô tả tài liệu \nNội dung tài liệu bao gồm các phần sau: \n1. \nMục A: Giới thiệu tổng quan \n2. \nMục B: Hướng dẫn các chức năng hệ thống có trên APP cho người dân sử \ndụng. \n1.3 \n Thuật ngữ viết tắt \nSTT \nThuật ngữ \nÝ nghĩa \n1 \nCCCD \nCăn cước công dân \n2 \nSĐT \nSố điện thoại \n3 \nNSD \nNgười sử dụng \n1.4 Cấu trúc hệ thống \nSau khi đăng nhập vào hệ thống, màn hình trang chủ hiển thị giao diện như hình.',
 '8 \n \n \nHình 1 Giao diện trang chủ mức 0 \n1.5 Chức năng chung \n-  Đăng nhập \n-  Đăng ký mức 0 \n-  Quên mật khẩu \n-  Đăng ký mức 1 \n-  Kích hoạt tài khoản \n-  Trang chủ  \n-  Ví giấy tờ  \n-  Tab Cá nhân \n- Đổi tài khoản \n- Thông báo lưu trú',
 '9 \n \n2. HƯỚNG DẪN SỬ DỤNG \n2.1 Hướng dẫn cài đặt  \n2.1.1  Đối với hệ điều hành Android \nCài đặt ứng dụng từ CH Play \n-    Bước 1: NS

In [29]:
import re
sentences = re.split(r'(?<=[.!?])\s+', source[0].strip())

In [30]:
sentences

['7 \n \n1.',
 'GIỚI THIỆU TỔNG QUAN \n1.1 \nĐối tượng sử dụng \n- \nDùng cho công dân Việt Nam có căn cước công dân gắn chíp thực hiện \nđăng ký tài khoản Định danh diện tử \n1.2 \n Mô tả tài liệu \nNội dung tài liệu bao gồm các phần sau: \n1.',
 'Mục A: Giới thiệu tổng quan \n2.',
 'Mục B: Hướng dẫn các chức năng hệ thống có trên APP cho người dân sử \ndụng.',
 '1.3 \n Thuật ngữ viết tắt \nSTT \nThuật ngữ \nÝ nghĩa \n1 \nCCCD \nCăn cước công dân \n2 \nSĐT \nSố điện thoại \n3 \nNSD \nNgười sử dụng \n1.4 Cấu trúc hệ thống \nSau khi đăng nhập vào hệ thống, màn hình trang chủ hiển thị giao diện như hình.']

In [31]:
[len(sen) for sen in sentences]

[7, 219, 31, 75, 227]